In [1]:
#get subset of MUSC records where journals equal those two chosen.

import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element
import pickle
import os
import pandas as pd
import numpy as np
import nltk.data

In [2]:
os.chdir('/home/sarahwie/Documents/pubmed-nlp-research/output/')
#os.chdir('/mnt/mypartition/Desktop2/')

In [3]:
#***WOULD NEED to bring this file over to the VM
import sys
sys.path.append('/home/sarahwie/Documents/pubmed-nlp-research/DeepLearningMovies_datasets/')
from KaggleWord2VecUtility import KaggleWord2VecUtility
DATADIR='/home/sarahwie/Documents/pubmed-nlp-research/DeepLearningMovies_datasets/'

In [4]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

### Getting subset based on 2 journals (with abstracts and no Unicode encoding)

In [6]:
tree = ET.parse('/mnt/mypartition/Desktop2/testXMLwrite.xml')
root = tree.getroot()

In [7]:
#using this file for our example
xml_file = ET.parse('/home/sarahwie/Documents/zip/subset/zip/medline16n0189.xml')
root = xml_file.getroot()

This code subsets the records down to those with the titles we are looking for (as well as those with abstracts).

In [8]:
#Can delete I think for real run
#make new XML file
rootNew = Element('MedlineCitationSet')
children = []

In [9]:
#can delete I think for real run
#replace titles here with those we're using
titles = ['The Journal of biological chemistry', 'Journal of bacteriology']

i = 0
for record in root.findall('MedlineCitation'):
    title = record.find('Article').find('Journal').find('Title').text
    
    if title in titles:
        #for test run, also check if abstract
        try:
            abstract = record.find('Article').find('Abstract').find('AbstractText').text
            if type(abstract) != str:
                print i
            i = i + 1
            #append to subset file
            children.append(record)
        except AttributeError:
            #dont include if no abstract
            #something random to hold place of indent:
            x = 0

rootNew.extend(children)

#create tree and write to file
tree = ET.ElementTree(rootNew)
tree.write("subsetTwoJournals.xml")
        
pickle.dump(rootNew, open('rootTwoJournals.p', 'wb'))

152
363


In [10]:
#Can delete I think for real run
#size of new subset
print len(rootNew.getchildren())

422


/home/sarahwie/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  app.launch_new_instance()


### Getting training and testing sets as pandas dataframes

In [11]:
#convert XML subset to pandas dataframe
def iter_docs(author):
    author_attr = author.attrib
    for record in author.findall('MedlineCitation'):
        doc_dict = author_attr.copy()
        #doc_dict.update(record.attrib)
        doc_dict['journal'] = record.find('Article').find('Journal').find('Title').text
        abstract = record.find('Article').find('Abstract').find('AbstractText').text
        #make sure all abstracts are in string format to avoid later sentence parsing issues (two in test run were unicode)
        if type(abstract) != str:
            abstract = abstract.encode('utf8')
        doc_dict['abstract'] = abstract
        yield doc_dict
        
df = pd.DataFrame(list(iter_docs(rootNew)))
df = df[['journal', 'abstract']]
print df.iloc[363:364,]

                                 journal  \
363  The Journal of biological chemistry   

                                              abstract  
363  Metallothionein was purified under anaerobic c...  


In [13]:
print df.loc[1][0]
#don't know why no quotation marks like when using pandas.to_csv in original w2v inversion
#but seems to work fine so ignore for now
df.loc[0:2]

Journal of bacteriology


,journal,abstract
0,Journal of bacteriology,ompB mutants of Escherichia coli K-12 are mark...
1,Journal of bacteriology,Several lysosomal glycosidase activities were ...
2,Journal of bacteriology,The initiation of patulin biosynthesis in subm...


In [14]:
#split into train and test
fracTrain = 0.75
nSamples = df.shape[0]
order = np.random.permutation(nSamples) # come up with a random ordering
splitIndex = int(np.round(nSamples*fracTrain))
train1 = df.ix[order[:splitIndex],:]
test1 = df.ix[order[splitIndex:],:]

In [15]:
print train1.shape
print test1.shape

(316, 2)
(106, 2)


In [17]:
#print first record in test set (note has different index value, indexed instead by location (pos. 0))
print test1.iloc[[0]]

                                 journal  \
371  The Journal of biological chemistry   

                                              abstract  
371  An icosadeoxyribonucleotide containing the sev...  


### Parsing Sentences & Building Word2Vec Model

Some testing:

In [197]:
review = train1["abstract"].iloc[0]
review = review.encode('utf8')
review.decode('utf8')
#KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

#flatten list
#[item for sublist in lst for item in sublist]

u"Most strains of Escherichia coli K-12 are unable to use the enzyme IIA/IIB (enzyme IIMan) complex of the phosphoenolpyruvate:sugar phosphotransferase system (PTS) in anaerobic growth and therefore cannot utilize glucosamine anaerobically. Introduction into these strains of a ptsG mutation, which eliminates activity of the enzyme IIIGlc/IIB' complex of the PTS, resulted in inability to grow anaerobically on glucose and mannose. Derivative strains able to grow anaerobically on glucosamine had mutations at a locus close to man, the gene coding for phosphomannose isomerase, and had higher enzyme IIA/IIB activities during anaerobic growth than did the parental strain. These results establish a locus affecting function of enzyme IIA/IIB that maps distant from ptsM, the probable structural gene for enzyme IIB."

Gets a list of positive review indexes and negative review indexes. Takes those respective reviews and parses them into sentences and appends to a list of positive and negative sentences, respectively. Also does the same for the unlabelled training sentences.

In [20]:
# ****** Split the training set into clean sentences
#
sentences_pos = []  # Initialize an empty list of sentences
sentences_neg = []  # Initialize an empty list of sentences

#here change to include all journal name labels of positive and negative
inxs_pos = np.where(train1["journal"] == 'The Journal of biological chemistry')[0].tolist()
inxs_neg = np.where(train1["journal"] == 'Journal of bacteriology')[0].tolist()

print "Parsing sentences from training set"
for inx in inxs_pos:
    review = train1["abstract"].iloc[inx]
    sentences_pos += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

for inx in inxs_neg:
    review = train1["abstract"].iloc[inx]
    sentences_neg += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)    

Parsing sentences from training set


In [22]:
print sentences_pos[0:2]

[[u'proteolysis', u'of', u'nereis', u'cuticle', u'collagen', u'by', u'two', u'bacterial', u'collagenases', u'was', u'investigated', u'using', u'viscosimetry', u'enzyme', u'kinetics', u'sodium', u'dodecyl', u'sulfate', u'polyacrylamide', u'gel', u'electrophoresis', u'and', u'ion', u'exchange', u'chromatography', u'of', u'collagenolytic', u'peptides'], [u'collagenase', u'of', u'the', u'marine', u'vibrio', u'b', u'completely', u'degrades', u'native', u'cuticle', u'collagen', u'at', u'degress', u'c', u'with', u'a', u'turnover', u'number', u'times', u'greater', u'than', u'that', u'of', u'the', u'clostridial', u'collagenase']]


Now do the same with all sentences to build the vocabulary

In [23]:
# ****** Split the labeled and unlabeled training sets into clean sentences
#
sentences = []  # Initialize an empty list of sentences

print "Parsing sentences from training set"
for review in train1["abstract"]:
    sentences += KaggleWord2VecUtility.review_to_sentences(review, tokenizer)

Parsing sentences from training set


Build Word2Vec model

In [24]:
from gensim.models import Word2Vec
import multiprocessing

## create a w2v learner 
basemodel = Word2Vec(
    workers=multiprocessing.cpu_count(), # use your cores
    iter=3, # iter = sweeps of SGD through the data; more is better
    hs=1, negative=0 # we only have scoring for the hierarchical softmax setup
    )
print basemodel

Word2Vec(vocab=0, size=100, alpha=0.025)


In [25]:
basemodel.build_vocab(sentences) 

Train Word2Vec model:

In [27]:
#model 0 corresponds with the Journal of bacteriology
#and model 1 the Journal of Biological Chemistry

from copy import deepcopy
models = [deepcopy(basemodel) for i in range(2)]
models[0].train(sentences_neg, total_examples=len(sentences_neg) )
models[1].train(sentences_pos, total_examples=len(sentences_pos) )

53286

### Applying inversion on the test set

In [28]:
"""
docprob takes two lists
* docs: a list of documents, each of which is a list of sentences
* models: the candidate word2vec models (each potential class)

it returns the array of class probabilities.  Everything is done in-memory.
"""

import pandas as pd # for quick summing within doc

def docprob(docs, mods):
    # score() takes a list [s] of sentences here; could also be a sentence generator
    sentlist = [s for d in docs for s in d]
    # the log likelihood of each sentence in this review under each w2v representation
    llhd = np.array( [ m.score(sentlist, len(sentlist)) for m in mods ] )
    # now exponentiate to get likelihoods, 
    lhd = np.exp(llhd - llhd.max(axis=0)) # subtract row max to avoid numeric overload
    # normalize across models (stars) to get sentence-star probabilities
    #all this transposing business does is make it so the total probability of a word 
    #   equals 1 between the 2 arrays (positive prob and negative).
    #and the pandas data frame just puts everything into rows/columns format for easy viz
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    # and finally average the sentence probabilities to get the review probability
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    prob = prob.groupby("doc").mean()
    return prob

In [29]:
# read in the test set as a list of a list of words
docs = []
for review in test1["abstract"]:
    docs.append(KaggleWord2VecUtility.review_to_sentences(review, tokenizer))

In [30]:
print docs[0]

[[u'an', u'icosadeoxyribonucleotide', u'containing', u'the', u'several', u'features', u'found', u'in', u'prokaryotic', u'mrna', u'ribosome', u'binding', u'sites', u'has', u'been', u'synthesized'], [u'this', u'sequence', u'can', u'stimulate', u'the', u'binding', u'of', u'initiator', u'fmet', u'trnaf', u'to', u'the', u'ribosome', u'to', u'form', u'a', u'stable', u's', u'initiation', u'complex', u'identical', u'with', u'those', u'induced', u'by', u'natural', u'messengers'], [u'the', u'binding', u'of', u'this', u'synthetic', u'ribosome', u'binding', u'site', u'is', u'absolutely', u'dependent', u'upon', u'initiation', u'factor', u'if', u'and', u'the', u'bound', u'fmet', u'trnaf', u'is', u'sensitive', u'to', u'puromycin', u'indicating', u'the', u'formation', u'of', u'a', u'functional', u'initiation', u'complex'], [u'a', u'heptadecadeoxyribonucleotide', u'identical', u'with', u'the', u'icosanucleotide', u'but', u'lacking', u'the', u'terminal', u'a', u't', u'g', u'codon', u'can', u'also', u'st

In [31]:
# get the probs (note we give docprob our test set plus the models)
probs = docprob(docs,models).astype(object)

In [32]:
probs

,0,1
doc,,
0,0.847698,0.152302
1,0.971511,0.0284893
2,0.800854,0.199146
3,0.748073,0.251927
4,0.764333,0.235667
5,0.856744,0.143256
6,0.933766,0.066234
7,0.768516,0.231484
8,0.992793,0.00720701


In [33]:
predictions = np.ones((probs.shape[0]))

In [34]:
print np.shape(predictions)

(106,)


In [35]:
predictions[np.where(probs.iloc[:,0] > 0.5)] = 0 # The first column is the negative model

In [36]:
predictions

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.])

In [ ]:
print predictions.shape
print test1["sentiment"].shape

In [ ]:
print np.size(np.where(predictions == test1["sentiment"]))*1./np.size(predictions)